In [23]:
import os
import re
import pandas as pd
import pickle
import collections
import numpy as np
import math
from ast import literal_eval
from time import gmtime, strftime
import re
import time
from bs4 import BeautifulSoup
from tqdm import trange, tqdm_notebook
from tqdm.auto import tqdm

# Scrapping
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent

# Error Handling
import socket
import urllib3
import urllib.request
from urllib.request import urlopen
from urllib.parse import quote_plus
from urllib.request import urlretrieve
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, ElementClickInterceptedException
import warnings
warnings.filterwarnings('ignore')

from collections import defaultdict

In [100]:
v4 = pd.read_csv("v4_update.csv")
v4 = v4.iloc[:, 1:]

In [ ]:
v4_update_dup_brd_lst = list(set(list(v4.loc[v4.duplicated(['제품 link']), '브랜드명'])))
v4_update_dup_prd_lst = list(set(list(v4.loc[v4.duplicated(['제품 link']), '제품명'])))

# v4_3에서 이미 중복되어 있는 상품 삭제

In [91]:
v4_3 = pd.read_csv("glowpick_product_info_v4.3.csv")
v4_3 = v4_3.iloc[:, 1:]

In [92]:
len(v4_3)

63465

In [93]:
v4_3_dup_lst = list(v4_3.loc[v4_3.duplicated(['제품_id']), '제품_id'])

In [99]:
len(v4_3_dup_lst)

40

In [95]:
len(v4)

64282

In [96]:
drop_idx = []

In [102]:
for i in range(len(v4)):
    if v4.loc[i, "제품 id"] in v4_3_dup_lst:
        drop_idx.append(i)
        v4_3_dup_lst.remove(v4.loc[i, "제품 id"])

In [104]:
len(drop_idx)

40

In [105]:
v4 = v4.drop(drop_idx)

In [106]:
len(v4)

64243

In [248]:
v4 = v4.iloc[:, 2:]

In [250]:
v4.to_csv("v4_update.csv")

In [167]:
import pickle

with open("v4_update_drop_duplicates.csv", "wb") as f:
    pickle.dump(v4, f)

In [168]:
import pickle
with open("v4_update_drop_duplicates.csv", "rb") as f:
    v4 = pickle.load(f)

In [255]:
v4.loc[30486]

브랜드 link                                                                     
브랜드명                                                                     끌라삐엘
브랜드 id                                                                    NaN
제품 link                               https://www.glowpick.com/products/89100
제품 id                                                                   89100
제품명                                                                    올인원 크림
제품 용량                                                                     50g
제품 가격                                                                 82,000원
유사 제품군 랭킹(int)                                                             []
유사 제품군 랭킹(text)                                                            []
브랜드 내 랭킹(int)                                                              []
제품 수상명                                                                     []
제품 수상 부문                                                        

In [235]:
v4_dup_id_lst = list(v4.loc[v4.duplicated(['제품 id']), '제품 id'])
v4_dup_name_lst = list(v4.loc[v4.duplicated(['제품 id']), '제품명'])

In [240]:
df_v4_dup = pd.DataFrame()

In [242]:
idx = 0
for i in range(len(v4)):
    if v4.loc[i, '제품 id'] in v4_dup_id_lst:
        df_v4_dup[idx] = v4.loc[i, :]
        idx += 1

C:\Users\keunb\AppData\Local\Temp/ipykernel_6600/589352488.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df_v4_dup[idx] = v4.loc[i, :]


In [244]:
df_v4_dup = df_v4_dup.transpose()

In [246]:
df_v4_dup.to_csv("df_v4_dup.csv")

In [252]:
drop_idx = []

---
# find_missing_df_brd_w_prd_v2_drop_lg_3220

In [15]:
df_brd_w_prd_v2_drop_lg_3220 = pd.read_csv("find_missing_df_brd_w_prd_v1_drop_lg_3220.csv")
df_brd_w_prd_v2_drop_lg_3220 = df_brd_w_prd_v2_drop_lg_3220.iloc[:3220, 1:]

In [17]:
exception = list(df_brd_w_prd_v2_drop_lg_3220.loc[np.isnan(df_brd_w_prd_v2_drop_lg_3220["prd_len"]), :].index)

In [24]:
def get_search_len(brd_name):

    try: 
        options = Options()
        ua = UserAgent()
        userAgent = ua.random
        print(userAgent)

        options.add_argument('headless')
        options.add_argument('window-size=1920x1080')
        options.add_argument("disable-gpu")
        options.add_argument(f'user-agent={userAgent}')
        wd = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)

        socket.setdefaulttimeout(30)

        url = 'https:/www.glowpick.com/searches/keywords'
        wd.get(url)
        time.sleep(5)
        
        html = wd.page_source
        soup = BeautifulSoup(html, "html.parser")

        wd.find_element_by_xpath("//*[@id='default-layout']/div/div[1]/span/div/div[2]/div[2]/button[2]").click()
        wd.maximize_window()
        time.sleep(3)

        brd_input = wd.find_element_by_xpath("//*[@id='default-layout']/div/div[2]/div/div/div/input")
        brd_input.send_keys(brd_name)
        brd_input.send_keys(Keys.ENTER)
        time.sleep(3)
        
        # ====

        #.products__product
        #.product
        # ====

        def parsing_html_with_class(parsing_info = '.products'):
            html = wd.page_source
            soup = BeautifulSoup(html, "html.parser")
            interm_r = soup.select(parsing_info)
            return interm_r
        
        try:
            while True:

                product_lst = parsing_html_with_class()
                
                global product_len
                product_len = len(re.findall('<div class="products__product product" (.*?)', str(product_lst[0])))
                

                element = wd.find_element_by_xpath('/html/body/div/div/div/div/main/div/section/div[2]/div/div[1]/span/div[{}]'.format(product_len - 1))
                wd.execute_script("arguments[0].scrollIntoView(true);", element)
                time.sleep(2)

                product_lst_2 = parsing_html_with_class()

                if product_lst == product_lst_2:
                    html = wd.page_source
                    soup = BeautifulSoup(html, "html.parser")
                    prd_len = len(soup.find_all('div','products__product product'))
                    return prd_len
                    break


        except NoSuchElementException:
            html = wd.page_source
            soup = BeautifulSoup(html, "html.parser")
            prd_len = len(soup.find_all('div','products__product product'))
            return prd_len
        
    except Exception as e:
        exception.append(brd_name)
        print("brand name: {}".format(brd_name), e)
        pass

In [19]:
exception

[94,
 109,
 210,
 269,
 296,
 333,
 373,
 384,
 524,
 579,
 722,
 745,
 746,
 801,
 823,
 853,
 860,
 880,
 984,
 1012,
 1233,
 1272,
 1296,
 1333,
 1390,
 1405,
 1413,
 1560,
 1581,
 1804,
 1862,
 1997,
 2113,
 2115,
 2189,
 2209,
 2290,
 2380,
 2437,
 2455,
 2459,
 2579,
 2620,
 2651,
 2707,
 2718,
 2728,
 2742,
 2773,
 2822,
 2824,
 2995,
 3133,
 3179,
 3182,
 3207]

In [26]:
df_brd_w_prd_v2_drop_lg_3220

,brand_code_x,brand_name,product_link,product_name,brand_code_y,category,crawling_prd_cnt,prd_len
0,https://www.glowpick.com/brands/8016,가가알로에,['https://www.glowpick.com/products/104713'],['라율 나이트 케어 너리싱 젤'],https://www.glowpick.com/brands/8016,['스킨케어'],1.0,1.0
1,https://www.glowpick.com/brands/2077,가네보,"['https://www.glowpick.com/products/101993', '...","['페이스업 파우더 밀라노 콜렉션 2018 [SPF14/PA+]', '코프레도르 스...",https://www.glowpick.com/brands/2077,"['페이스메이크업', '립메이크업', '컨투어링', '클렌징', '배쓰&바디', '...",12.0,12.0
2,https://www.glowpick.com/brands/8724,가드미,"['https://www.glowpick.com/products/112824', '...","['큐티클 클리너', '네일 큐티클 호호바 오일', '[단종] 슈퍼 단백질 헤어가...",https://www.glowpick.com/brands/8724,['네일'],3.0,3.0
3,https://www.glowpick.com/brands/10352,가든 오브 라이프,"['https://www.glowpick.com/products/133823', '...","['비타민 로우 B 컴플렉스', '로우 프로바이오틱스 장케어', '무가공 비타민C'...",https://www.glowpick.com/brands/10352,['이너뷰티'],12.0,12.0
4,https://www.glowpick.com/brands/7314,가디스가든,"['https://www.glowpick.com/products/96142', 'h...","['유기농 드림 리페어 선 리페어 나이트 크림', '유기농 데이 언던 선 리페어 세...",https://www.glowpick.com/brands/7314,"['스킨케어', '베이비', '립메이크업', '클렌징', '선케어']",14.0,14.0
...,...,...,...,...,...,...,...,...
3215,https://www.glowpick.com/brands/8150,산 미니아또 알 몬떼,['https://www.glowpick.com/products/106529'],['https://www.glowpick.com/products/106529'],https://www.glowpick.com/brands/8150,['헤어'],1.0,1.0
3216,https://www.glowpick.com/brands/6021,산다화,"['https://www.glowpick.com/products/127617', '...","['https://www.glowpick.com/products/127617', '...",https://www.glowpick.com/brands/6021,"['스킨케어', '페이스메이크업', '베이비', '립메이크업', '남성화장품', '...",59.0,59.0
3217,https://www.glowpick.com/brands/11597,산도깨비,['https://www.glowpick.com/products/144794'],['https://www.glowpick.com/products/144794'],https://www.glowpick.com/brands/11597,['클렌징'],1.0,1.0
3218,https://www.glowpick.com/brands/10214,산듯,"['https://www.glowpick.com/products/132170', '...","['https://www.glowpick.com/products/132170', '...",https://www.glowpick.com/brands/10214,['스킨케어'],3.0,3.0


In [69]:
df_brd_w_prd_v2_drop_lg_3220.loc[np.isnan(df_brd_w_prd_v2_drop_lg_3220["prd_len"])].index

Int64Index([], dtype='int64')

In [67]:
df_brd_w_prd_v2_drop_lg_3220.loc[2707, :]

brand_code_x               https://www.glowpick.com/brands/11207
brand_name                                                  벨앙브아
product_link        ['https://www.glowpick.com/products/141224']
product_name                                        ['콜라겐 마스크팩']
brand_code_y               https://www.glowpick.com/brands/11207
category                                               ['마스크/팩']
crawling_prd_cnt                                             1.0
prd_len                                                      NaN
Name: 2707, dtype: object

In [68]:
for i in tqdm(list(df_brd_w_prd_v2_drop_lg_3220.loc[np.isnan(df_brd_w_prd_v2_drop_lg_3220["prd_len"])].index)):
    brd_name = df_brd_w_prd_v2_drop_lg_3220.loc[i, 'brand_name']
    prd_len = get_search_len(brd_name)
    df_brd_w_prd_v2_drop_lg_3220.loc[i, 'prd_len'] = prd_len

  0%|          | 0/1 [00:00<?, ?it/s]



====== WebDriver manager ======


Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/30.0.1599.17 Safari/537.36


Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606
Driver [C:\Users\keunb\.wdm\drivers\chromedriver\win32\94.0.4606.61\chromedriver.exe] found in cache


In [94]:
df_brd_w_prd_v2_drop_lg_3220.to_csv("find_missing_df_brw_w_prd_v1_drop_lg_3220.csv")

In [87]:
add_prd = 0

for i in list(df_brd_w_prd_v2_drop_lg_3220.loc[df_brd_w_prd_v2_drop_lg_3220['crawling_prd_cnt'] < df_brd_w_prd_v2_drop_lg_3220['prd_len']].index):
    add_prd += df_brd_w_prd_v2_drop_lg_3220.loc[i, 'prd_len'] - df_brd_w_prd_v2_drop_lg_3220.loc[i, 'crawling_prd_cnt']

In [88]:
add_prd

291.0

In [80]:
del_prd = 0

for i in list(df_brd_w_prd_v2_drop_lg_3220.loc[df_brd_w_prd_v2_drop_lg_3220['crawling_prd_cnt'] > df_brd_w_prd_v2_drop_lg_3220['prd_len']].index):
    del_prd += df_brd_w_prd_v2_drop_lg_3220.loc[i, 'crawling_prd_cnt'] - df_brd_w_prd_v2_drop_lg_3220.loc[i, 'prd_len']

In [81]:
del_prd

895.0

In [97]:
df_brd_w_prd_v2_drop_lg_3220.loc[df_brd_w_prd_v2_drop_lg_3220['crawling_prd_cnt'] < df_brd_w_prd_v2_drop_lg_3220['prd_len']]

,brand_code_x,brand_name,product_link,product_name,brand_code_y,category,crawling_prd_cnt,prd_len
32,https://www.glowpick.com/brands/7406,고려은단,"['https://www.glowpick.com/products/97645', 'h...","['비타민C 1000', '비타민C 골드 플러스', '비타민C 1000 이지', '...",https://www.glowpick.com/brands/7406,"['이너뷰티', '마스크/팩']",17.0,18.0
152,https://www.glowpick.com/brands/11349,글로우,"['https://www.glowpick.com/products/142634', '...","['식이섬유 핑크 스틱', '콜라겐 퍼플 스틱', '면역 옐로우 스틱', '컬러 푸...",https://www.glowpick.com/brands/11349,['이너뷰티'],1137.0,1140.0
163,https://www.glowpick.com/brands/10911,글린트 바이 비디보브,"['https://www.glowpick.com/products/139568', '...","['하이라이터', '아이팔레트', '글리터 젤', '스파클 립스틱', '쉬어 립스틱...",https://www.glowpick.com/brands/10911,"['페이스메이크업', '립메이크업', '아이메이크업', '컨투어링']",6.0,7.0
208,https://www.glowpick.com/brands/9974,끌,"['https://www.glowpick.com/products/128543', '...","['리프팅 마스크', '시그니처 오 드 뚜왈렛', '끌로에 오 드 퍼퓸', '에빠끌...",https://www.glowpick.com/brands/9974,['마스크/팩'],446.0,450.0
209,https://www.glowpick.com/brands/6189,끌라삐엘,"['https://www.glowpick.com/products/89101', 'h...","['리치 아이크림', '보건용 황사방역 마스크 KF94 대형', '히비스커스 릴렉싱...",https://www.glowpick.com/brands/6189,"['스킨케어', '페이스메이크업', '립메이크업', '클렌징', '마스크/팩', '...",18.0,22.0
...,...,...,...,...,...,...,...,...
3049,https://www.glowpick.com/brands/10895,비아르랩,"['https://www.glowpick.com/products/148582', '...","['https://www.glowpick.com/products/148582', '...",https://www.glowpick.com/brands/10895,"['스킨케어', '마스크/팩']",10.0,11.0
3149,https://www.glowpick.com/brands/8710,비플레인,"['https://www.glowpick.com/products/124509', '...","['https://www.glowpick.com/products/124509', '...",https://www.glowpick.com/brands/8710,"['스킨케어', '클렌징', '마스크/팩', '선케어']",23.0,24.0
3154,https://www.glowpick.com/brands/9525,빅,"['https://www.glowpick.com/products/122212', '...","['https://www.glowpick.com/products/122212', '...",https://www.glowpick.com/brands/9525,['배쓰&바디'],334.0,335.0
3172,https://www.glowpick.com/brands/1486,빠이요,"['https://www.glowpick.com/products/73787', 'h...","['https://www.glowpick.com/products/73787', 'h...",https://www.glowpick.com/brands/1486,"['스킨케어', '립메이크업', '클렌징', '마스크/팩', '선케어', '배쓰&바디']",20.0,21.0
